# 2024 MCM C题 - 温网比赛数据预处理

## 背景
本题需要分析2023年温布尔登网球锦标赛男子单打比赛的逐分数据，研究比赛中的"势头"现象。

## 预处理目标
1. 加载并检查数据质量
2. 处理缺失值和异常值
3. 转换数据类型
4. 创建势头分析相关的特征
5. 保存清洗后的数据

## 第一步：数据加载与初探

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# 加载主数据集
df = pd.read_csv('2024_Wimbledon_featured_matches.csv')

# 加载数据字典
data_dict = pd.read_csv('2024_data_dictionary.csv')

print("="*60)
print("主数据集基本信息")
print("="*60)
print(f"数据形状: {df.shape[0]} 行 x {df.shape[1]} 列")
print(f"\n列名列表:")
print(df.columns.tolist())

主数据集基本信息
数据形状: 7284 行 x 46 列

列名列表:
['match_id', 'player1', 'player2', 'elapsed_time', 'set_no', 'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games', 'p1_score', 'p2_score', 'server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won', 'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner', 'p2_winner', 'winner_shot_type', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err', 'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won', 'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won', 'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed', 'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph', 'serve_width', 'serve_depth', 'return_depth']


In [3]:
# 查看数据字典
print("数据字典:")
data_dict

数据字典:


,variables,explanation,example
0,match_id,match identification,"2023-wimbledon-1701 (""7"" is the round, and ""01..."
1,player1,first and last name of the first player,Carlos Alcaraz
2,player2,first and last name of the second player,Novak Djokovic
3,elapsed_time,time elapsed since start of first point to sta...,0:10:27
4,set_no,set number in match,"1, 2, 3, 4, or 5"
5,game_no,game number in set,"1, 2, ...,7"
6,point_no,point number in game,"1, 2, 3... etc."
7,p1_sets,sets won by player 1,"0, 1, or 2"
8,p2_sets,sets won by player 2,"0, 1, or 2"
9,p1_games,games won by player 1 in current set,"0, 1,...,6"


In [4]:
# 查看数据类型和基本信息
print("\n数据类型信息:")
df.info()


数据类型信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7284 entries, 0 to 7283
Data columns (total 46 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   match_id            7284 non-null   object 
 1   player1             7284 non-null   object 
 2   player2             7284 non-null   object 
 3   elapsed_time        7284 non-null   object 
 4   set_no              7284 non-null   int64  
 5   game_no             7284 non-null   int64  
 6   point_no            7284 non-null   int64  
 7   p1_sets             7284 non-null   int64  
 8   p2_sets             7284 non-null   int64  
 9   p1_games            7284 non-null   int64  
 10  p2_games            7284 non-null   int64  
 11  p1_score            7284 non-null   object 
 12  p2_score            7284 non-null   object 
 13  server              7284 non-null   int64  
 14  serve_no            7284 non-null   int64  
 15  point_victor        7284 non-null   int64  
 1

In [5]:
# 查看前几行数据
print("\n数据预览 (前10行):")
df.head(10)


数据预览 (前10行):


,match_id,player1,player2,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,p2_games,p1_score,p2_score,server,serve_no,point_victor,p1_points_won,p2_points_won,game_victor,set_victor,p1_ace,p2_ace,p1_winner,p2_winner,winner_shot_type,p1_double_fault,p2_double_fault,p1_unf_err,p2_unf_err,p1_net_pt,p2_net_pt,p1_net_pt_won,p2_net_pt_won,p1_break_pt,p2_break_pt,p1_break_pt_won,p2_break_pt_won,p1_break_pt_missed,p2_break_pt_missed,p1_distance_run,p2_distance_run,rally_count,speed_mph,serve_width,serve_depth,return_depth
0,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:00,1,1,1,0,0,0,0,0,0,1,2,2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,6.00,7.84,2,95.00,BC,NCTL,ND
1,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:38,1,1,2,0,0,0,0,0,15,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.25,7.09,1,118.00,B,CTL,ND
2,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:01,1,1,3,0,0,0,0,15,15,1,1,2,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,13.80,19.81,4,120.00,B,NCTL,D
3,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:31,1,1,4,0,0,0,0,15,30,1,1,1,2,2,0,0,0,0,1,0,F,0,0,0,0,0,1,0,0,0,0,0,0,0,0,51.11,75.63,13,130.00,BW,CTL,D
4,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:02:21,1,1,5,0,0,0,0,30,30,1,1,1,3,2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.65,0.81,1,112.00,W,NCTL,NaN
5,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:02:50,1,1,6,0,0,0,0,40,30,1,2,2,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.29,4.25,2,97.00,BW,NCTL,ND
6,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:03:33,1,1,7,0,0,0,0,40,40,1,1,1,4,3,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.82,17.82,1,109.00,W,CTL,D
7,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:04:01,1,1,8,0,0,0,0,AD,40,1,2,2,4,4,0,0,0,0,0,1,F,0,0,0,0,0,1,0,1,0,0,0,0,0,0,17.50,18.07,6,105.00,B,NCTL,ND
8,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:04:48,1,1,9,0,0,0,0,40,40,1,1,1,5,4,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,34.49,24.59,7,128.00,BC,CTL,D
9,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:05:32,1,1,10,0,0,0,0,AD,40,1,1,1,6,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.23,27.43,5,110.00,B,NCTL,ND


In [6]:
# 查看比赛数量和选手信息
print("\n比赛统计:")
print(f"- 比赛总数: {df['match_id'].nunique()}")
print(f"- 得分点总数: {len(df)}")

# 查看所有比赛
matches = df.groupby('match_id').agg({
    'player1': 'first',
    'player2': 'first',
    'point_no': 'count',
    'set_no': 'max'
}).rename(columns={'point_no': 'total_points', 'set_no': 'total_sets'})

print(f"\n各场比赛概览:")
matches


比赛统计:
- 比赛总数: 31
- 得分点总数: 7284

各场比赛概览:


,player1,player2,total_points,total_sets
match_id,,,,
2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,300,4
2023-wimbledon-1302,Alexander Zverev,Matteo Berrettini,201,3
2023-wimbledon-1303,Frances Tiafoe,Grigor Dimitrov,134,3
2023-wimbledon-1304,Alejandro Davidovich Fokina,Holger Rune,337,5
2023-wimbledon-1305,Daniil Medvedev,Marton Fucsovics,246,4
2023-wimbledon-1306,Jiri Lehecka,Tommy Paul,332,5
2023-wimbledon-1307,Christopher Eubanks,Christopher O'Connell,232,3
2023-wimbledon-1308,Laslo Djere,Stefanos Tsitsipas,190,3
2023-wimbledon-1309,Jannik Sinner,Quentin Halys,213,4


## 第二步：数据质量检查与清洗

In [7]:
def data_quality_report(df, name="DataFrame"):
    """生成数据质量报告"""
    print(f"数据质量报告: {name}")
    print(f"形状: {df.shape[0]} 行 x {df.shape[1]} 列")
    print("="*60)
    
    # 缺失值统计
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({'缺失数': missing, '缺失率%': missing_pct})
    missing_df = missing_df[missing_df['缺失数'] > 0].sort_values('缺失率%', ascending=False)
    
    if len(missing_df) > 0:
        print(f"\n缺失值统计 ({len(missing_df)} 列有缺失):")
        print(missing_df)
    else:
        print("\n无缺失值")
    
    # 重复值统计
    dup_count = df.duplicated().sum()
    print(f"\n完全重复行数: {dup_count} ({dup_count/len(df)*100:.2f}%)")
    
    return missing_df

missing_report = data_quality_report(df, "Wimbledon Matches")

数据质量报告: Wimbledon Matches
形状: 7284 行 x 46 列

缺失值统计 (4 列有缺失):
               缺失数  缺失率%
return_depth  1309 17.97
speed_mph      752 10.32
serve_width     54  0.74
serve_depth     54  0.74

完全重复行数: 0 (0.00%)


In [8]:
# 检查缺失值的具体情况
print("缺失值分析:")
print("="*60)

# 检查return_depth缺失情况
if 'return_depth' in df.columns:
    return_null = df[df['return_depth'].isnull()]
    print(f"\nreturn_depth 缺失行数: {len(return_null)}")
    if len(return_null) > 0:
        # 检查是否与ace相关（ace时没有回球）
        ace_count = (return_null['p1_ace'] == 1).sum() + (return_null['p2_ace'] == 1).sum()
        print(f"  - 其中ace球: {ace_count}")

缺失值分析:

return_depth 缺失行数: 1309
  - 其中ace球: 610


### 2.1 处理缺失值

根据业务逻辑分析缺失值的含义并进行处理：
- `speed_mph`: 发球速度，缺失可能因为是二发或测量问题
- `serve_width/serve_depth/return_depth`: 发球/回球位置，缺失可能是双误或未击中
- `winner_shot_type`: 制胜球类型，缺失表示没有制胜球（通过非受迫性失误得分）

In [9]:
# 创建清洗后的数据副本
df_clean = df.copy()

# 1. winner_shot_type: 缺失表示"无制胜球"，填充为"None"
if 'winner_shot_type' in df_clean.columns:
    df_clean['winner_shot_type'] = df_clean['winner_shot_type'].fillna('None')
    print("winner_shot_type: 缺失值填充为 'None'（表示非制胜球得分）")

# 2. return_depth: 缺失可能是ace球（无回球），填充为"NA_ace"
if 'return_depth' in df_clean.columns:
    is_ace = (df_clean['p1_ace'] == 1) | (df_clean['p2_ace'] == 1)
    df_clean.loc[df_clean['return_depth'].isnull() & is_ace, 'return_depth'] = 'NA_ace'
    df_clean['return_depth'] = df_clean['return_depth'].fillna('Unknown')
    print("return_depth: ace球填充为 'NA_ace'，其他填充为 'Unknown'")

# 3. serve_width和serve_depth: 填充为Unknown
for col in ['serve_width', 'serve_depth']:
    if col in df_clean.columns and df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna('Unknown')
        print(f"{col}: 缺失值填充为 'Unknown'")

# 4. speed_mph: 用同一发球次序的中位数填充
if 'speed_mph' in df_clean.columns and df_clean['speed_mph'].isnull().sum() > 0:
    df_clean['speed_mph'] = df_clean.groupby('serve_no')['speed_mph'].transform(
        lambda x: x.fillna(x.median())
    )
    print("speed_mph: 按发球次序(serve_no)的中位数填充")

winner_shot_type: 缺失值填充为 'None'（表示非制胜球得分）
return_depth: ace球填充为 'NA_ace'，其他填充为 'Unknown'
serve_width: 缺失值填充为 'Unknown'
serve_depth: 缺失值填充为 'Unknown'
speed_mph: 按发球次序(serve_no)的中位数填充


In [10]:
# 验证缺失值处理结果
print("\n处理后的缺失值情况:")
remaining_missing = df_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
if len(remaining_missing) > 0:
    print(remaining_missing)
else:
    print("所有缺失值已处理完毕")


处理后的缺失值情况:
所有缺失值已处理完毕


### 2.2 数据类型转换

In [11]:
# 转换时间列
if 'elapsed_time' in df_clean.columns:
    df_clean['elapsed_time_td'] = pd.to_timedelta(df_clean['elapsed_time'])
    df_clean['elapsed_seconds'] = df_clean['elapsed_time_td'].dt.total_seconds()
    print("elapsed_time: 转换为秒数 (elapsed_seconds)")

# 确保数值列的类型正确
numeric_cols = ['set_no', 'game_no', 'point_no', 'p1_sets', 'p2_sets', 
                'p1_games', 'p2_games', 'server', 'serve_no', 'point_victor',
                'p1_points_won', 'p2_points_won', 'game_victor', 'set_victor',
                'p1_ace', 'p2_ace', 'p1_winner', 'p2_winner',
                'p1_double_fault', 'p2_double_fault',
                'p1_unf_err', 'p2_unf_err',
                'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won', 'p2_net_pt_won',
                'p1_break_pt', 'p2_break_pt', 
                'p1_break_pt_won', 'p2_break_pt_won',
                'p1_break_pt_missed', 'p2_break_pt_missed',
                'rally_count', 'speed_mph']

for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print("数值列类型转换完成")

elapsed_time: 转换为秒数 (elapsed_seconds)
数值列类型转换完成


### 2.3 异常值检测

In [12]:
# 检查数值型变量的分布和异常值
print("数值变量统计描述:")
numeric_summary = df_clean.select_dtypes(include=[np.number]).describe()
numeric_summary.T

数值变量统计描述:


,count,mean,std,min,25%,50%,75%,max
set_no,7284.00,2.47,1.19,1.00,1.00,2.00,3.00,5.00
game_no,7284.00,5.91,3.41,1.00,3.00,6.00,8.00,13.00
point_no,7284.00,125.87,80.29,1.00,59.00,118.00,182.00,337.00
p1_sets,7284.00,0.77,0.81,0.00,0.00,1.00,1.00,2.00
p2_sets,7284.00,0.70,0.73,0.00,0.00,1.00,1.00,2.00
p1_games,7284.00,2.50,1.85,0.00,1.00,2.00,4.00,6.00
p2_games,7284.00,2.42,1.81,0.00,1.00,2.00,4.00,6.00
server,7284.00,1.51,0.50,1.00,1.00,2.00,2.00,2.00
serve_no,7284.00,1.36,0.48,1.00,1.00,1.00,2.00,2.00
point_victor,7284.00,1.49,0.50,1.00,1.00,1.00,2.00,2.00


In [13]:
# 检查可能的异常值
print("异常值检测:")
print("="*60)

# 发球速度范围检查 (通常在80-150 mph之间)
if 'speed_mph' in df_clean.columns:
    speed = df_clean['speed_mph'].dropna()
    print(f"\nspeed_mph 范围: {speed.min():.0f} - {speed.max():.0f} mph")
    outliers = df_clean[(df_clean['speed_mph'] < 60) | (df_clean['speed_mph'] > 160)]
    if len(outliers) > 0:
        print(f"异常发球速度行数: {len(outliers)}")
    else:
        print("发球速度在合理范围内")

# 跑动距离检查
for col in ['p1_distance_run', 'p2_distance_run']:
    if col in df_clean.columns:
        dist = df_clean[col].dropna()
        print(f"\n{col} 范围: {dist.min():.1f} - {dist.max():.1f} 米")

异常值检测:

speed_mph 范围: 72 - 141 mph
发球速度在合理范围内

p1_distance_run 范围: 0.0 - 148.7 米

p2_distance_run 范围: 0.0 - 156.9 米


In [14]:
# 逻辑一致性检查
print("\n逻辑一致性检查:")
print("="*60)

# 1. point_victor应该是1或2
invalid_victor = df_clean[~df_clean['point_victor'].isin([1, 2])]
print(f"point_victor非1/2的行数: {len(invalid_victor)}")

# 2. server应该是1或2
invalid_server = df_clean[~df_clean['server'].isin([1, 2])]
print(f"server非1/2的行数: {len(invalid_server)}")

# 3. ace只能由发球方打出
p1_ace_not_serving = df_clean[(df_clean['p1_ace'] == 1) & (df_clean['server'] != 1)]
p2_ace_not_serving = df_clean[(df_clean['p2_ace'] == 1) & (df_clean['server'] != 2)]
print(f"p1 ace但非p1发球的行数: {len(p1_ace_not_serving)}")
print(f"p2 ace但非p2发球的行数: {len(p2_ace_not_serving)}")


逻辑一致性检查:
point_victor非1/2的行数: 0
server非1/2的行数: 0
p1 ace但非p1发球的行数: 0
p2 ace但非p2发球的行数: 0


## 第三步：特征工程

为势头分析创建相关特征，注意避免数据泄露（只使用历史数据）。

In [15]:
# 为每场比赛创建全局得分点索引
df_clean['global_point_idx'] = df_clean.groupby('match_id').cumcount() + 1
print("添加 global_point_idx: 每场比赛中的得分点序号")

添加 global_point_idx: 每场比赛中的得分点序号


In [16]:
# 计算累计得分差（p1视角）
df_clean['point_diff'] = df_clean['p1_points_won'] - df_clean['p2_points_won']
print("添加 point_diff: 累计得分差（p1 - p2）")

添加 point_diff: 累计得分差（p1 - p2）


In [17]:
# 计算最近N个得分点的胜率（滚动窗口特征）
# 注意：使用shift(1)避免数据泄露

def calc_rolling_win_rate(group, window=5):
    """计算滚动胜率，shift(1)排除当前得分点"""
    p1_wins = (group['point_victor'] == 1).astype(int)
    rolling_wins = p1_wins.rolling(window=window, min_periods=1).sum().shift(1)
    rolling_count = p1_wins.rolling(window=window, min_periods=1).count().shift(1)
    return rolling_wins / rolling_count

# 最近5分胜率
df_clean['p1_rolling_win_rate_5'] = df_clean.groupby('match_id', group_keys=False).apply(
    lambda x: calc_rolling_win_rate(x, window=5)
).reset_index(level=0, drop=True)

# 最近10分胜率
df_clean['p1_rolling_win_rate_10'] = df_clean.groupby('match_id', group_keys=False).apply(
    lambda x: calc_rolling_win_rate(x, window=10)
).reset_index(level=0, drop=True)

print("添加 p1_rolling_win_rate_5/10: 最近5/10分p1胜率（不含当前分）")

添加 p1_rolling_win_rate_5/10: 最近5/10分p1胜率（不含当前分）


In [18]:
# 计算连续得分（连胜streak）
def calc_streak(group):
    """计算当前连胜数"""
    victor = group['point_victor']
    p1_streak, p2_streak = [], []
    cur_p1, cur_p2 = 0, 0
    
    for v in victor:
        if v == 1:
            cur_p1 += 1
            cur_p2 = 0
        else:
            cur_p2 += 1
            cur_p1 = 0
        p1_streak.append(cur_p1)
        p2_streak.append(cur_p2)
    
    return pd.DataFrame({'p1_streak': p1_streak, 'p2_streak': p2_streak}, index=group.index)

streak_df = df_clean.groupby('match_id', group_keys=False).apply(calc_streak)
df_clean['p1_streak'] = streak_df['p1_streak']
df_clean['p2_streak'] = streak_df['p2_streak']

# 前一个得分点的连胜数（避免泄露）
df_clean['p1_streak_prev'] = df_clean.groupby('match_id')['p1_streak'].shift(1).fillna(0)
df_clean['p2_streak_prev'] = df_clean.groupby('match_id')['p2_streak'].shift(1).fillna(0)

print("添加 p1_streak/p2_streak: 连胜数（当前）")
print("添加 p1_streak_prev/p2_streak_prev: 连胜数（前一分）")

添加 p1_streak/p2_streak: 连胜数（当前）
添加 p1_streak_prev/p2_streak_prev: 连胜数（前一分）


In [19]:
# 发球方优势指标
df_clean['is_p1_serving'] = (df_clean['server'] == 1).astype(int)
df_clean['server_won'] = (df_clean['server'] == df_clean['point_victor']).astype(int)

print("添加 is_p1_serving: p1是否发球")
print("添加 server_won: 发球方是否赢得该分")

添加 is_p1_serving: p1是否发球
添加 server_won: 发球方是否赢得该分


In [20]:
# 比赛进程指标
df_clean['games_in_set'] = df_clean['p1_games'] + df_clean['p2_games']
df_clean['sets_played'] = df_clean['p1_sets'] + df_clean['p2_sets']

print("添加 games_in_set: 当前盘已完成局数")
print("添加 sets_played: 已完成盘数")

添加 games_in_set: 当前盘已完成局数
添加 sets_played: 已完成盘数


In [21]:
# 关键分指标
df_clean['is_break_point'] = ((df_clean['p1_break_pt'] == 1) | (df_clean['p2_break_pt'] == 1)).astype(int)

df_clean['is_key_point'] = (
    (df_clean['is_break_point'] == 1) |
    ((df_clean['p1_score'].astype(str) == '40') & (df_clean['p2_score'].astype(str) == '40')) |
    ((df_clean['p1_score'].astype(str) == 'AD') | (df_clean['p2_score'].astype(str) == 'AD'))
).astype(int)

print("添加 is_break_point: 是否为破发点")
print("添加 is_key_point: 是否为关键分（破发点/平分/占先）")

添加 is_break_point: 是否为破发点
添加 is_key_point: 是否为关键分（破发点/平分/占先）


In [22]:
# 得分点间隔时间（反映比赛节奏）
df_clean['point_duration'] = df_clean.groupby('match_id')['elapsed_seconds'].diff()
df_clean['point_duration'] = df_clean['point_duration'].fillna(0)

print("添加 point_duration: 与上一得分点的时间间隔（秒）")

添加 point_duration: 与上一得分点的时间间隔（秒）


In [23]:
# 验证新创建的特征
print("\n新增特征统计:")
new_features = ['global_point_idx', 'point_diff', 'p1_rolling_win_rate_5', 'p1_rolling_win_rate_10',
                'p1_streak', 'p2_streak', 'p1_streak_prev', 'p2_streak_prev',
                'is_p1_serving', 'server_won', 'games_in_set', 'sets_played',
                'is_break_point', 'is_key_point', 'point_duration']

df_clean[new_features].describe()


新增特征统计:


,global_point_idx,point_diff,p1_rolling_win_rate_5,p1_rolling_win_rate_10,p1_streak,p2_streak,p1_streak_prev,p2_streak_prev,is_p1_serving,server_won,games_in_set,sets_played,is_break_point,is_key_point,point_duration
count,7284.00,7284.00,7253.00,7253.00,7284.00,7284.00,7284.00,7284.00,7284.00,7284.00,7284.00,7284.00,7284.00,7284.00,7284.00
mean,125.87,1.15,0.51,0.51,1.07,0.99,1.06,0.98,0.49,0.67,4.91,1.47,0.07,0.17,57.63
std,80.29,9.57,0.23,0.15,1.45,1.37,1.45,1.37,0.50,0.47,3.41,1.19,0.25,0.37,1024.50
min,1.00,-36.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,59.00,-4.00,0.40,0.40,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,26.00
50%,118.00,1.00,0.60,0.50,1.00,0.00,1.00,0.00,0.00,1.00,5.00,1.00,0.00,0.00,34.00
75%,182.00,7.00,0.60,0.60,2.00,2.00,2.00,2.00,1.00,1.00,7.00,2.00,0.00,0.00,46.00
max,337.00,30.00,1.00,1.00,13.00,11.00,13.00,11.00,1.00,1.00,12.00,4.00,1.00,1.00,86532.00


## 第四步：数据验证与保存

In [24]:
# 最终数据质量检查
print("="*60)
print("最终数据质量检查")
print("="*60)

print(f"\n最终数据形状: {df_clean.shape[0]} 行 x {df_clean.shape[1]} 列")

# 检查剩余缺失值
remaining_null = df_clean.isnull().sum()
remaining_null = remaining_null[remaining_null > 0]
if len(remaining_null) > 0:
    print(f"\n剩余缺失值:")
    print(remaining_null)
else:
    print("\n无缺失值")

# 检查数据类型
print(f"\n数据类型分布:")
print(df_clean.dtypes.value_counts())

最终数据质量检查

最终数据形状: 7284 行 x 63 列

剩余缺失值:
p1_rolling_win_rate_5     31
p1_rolling_win_rate_10    31
dtype: int64

数据类型分布:
int64              43
object             10
float64             9
timedelta64[ns]     1
Name: count, dtype: int64


In [25]:
# 查看处理后的数据示例（以决赛为例）
final_match = df_clean[df_clean['match_id'] == '2023-wimbledon-1701']
print(f"\n决赛数据预览 (2023-wimbledon-1701):")
print(f"- 总得分点数: {len(final_match)}")
print(f"- 选手: {final_match['player1'].iloc[0]} vs {final_match['player2'].iloc[0]}")

# 显示关键特征
display_cols = ['global_point_idx', 'set_no', 'game_no', 'point_no', 
                'p1_score', 'p2_score', 'server', 'point_victor',
                'point_diff', 'p1_streak', 'p2_streak', 'is_key_point']
final_match[display_cols].head(20)


决赛数据预览 (2023-wimbledon-1701):
- 总得分点数: 334
- 选手: Carlos Alcaraz vs Novak Djokovic


,global_point_idx,set_no,game_no,point_no,p1_score,p2_score,server,point_victor,point_diff,p1_streak,p2_streak,is_key_point
6950,1,1,1,1,0,0,2,2,-1,0,1,0
6951,2,1,1,2,0,15,2,1,0,1,0,0
6952,3,1,1,3,15,15,2,1,1,2,0,0
6953,4,1,1,4,30,15,2,2,0,0,1,0
6954,5,1,1,5,30,30,2,1,1,1,0,0
6955,6,1,1,6,40,30,2,2,0,0,1,1
6956,7,1,1,7,40,40,2,2,-1,0,2,1
6957,8,1,1,8,40,AD,2,1,0,1,0,1
6958,9,1,1,9,40,40,2,2,-1,0,1,1
6959,10,1,1,10,40,AD,2,2,-2,0,2,1


In [26]:
# 保存清洗后的数据
output_file = 'processed_wimbledon_data.csv'
df_clean.to_csv(output_file, index=False)
print(f"\n数据已保存至: {output_file}")
print(f"文件大小: {df_clean.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")


数据已保存至: processed_wimbledon_data.csv
文件大小: 7.38 MB


## 预处理总结

### 数据概况
- 原始数据包含2023年温网男单比赛的逐分记录
- 每行代表一个得分点，包含比赛状态、得分方式、选手表现等信息

### 清洗操作
1. **缺失值处理**:
   - `winner_shot_type`: 填充为'None'（表示非制胜球得分）
   - `return_depth`: ace球填充为'NA_ace'，其他填充为'Unknown'
   - `speed_mph`: 按发球次序的中位数填充
   
2. **数据类型转换**:
   - 时间转换为秒数便于计算
   - 分类变量转换为category类型

### 新增特征（用于势头分析）
| 特征名 | 说明 |
|--------|------|
| global_point_idx | 比赛中的得分点序号 |
| point_diff | 累计得分差(p1-p2) |
| p1_rolling_win_rate_5/10 | 最近5/10分p1胜率 |
| p1_streak/p2_streak | 连胜数 |
| is_p1_serving | p1是否发球 |
| server_won | 发球方是否赢 |
| is_break_point | 是否破发点 |
| is_key_point | 是否关键分 |
| point_duration | 得分点间隔时间 |

In [27]:
# 最终列清单
print("处理后数据的所有列:")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col}")

处理后数据的所有列:
 1. match_id
 2. player1
 3. player2
 4. elapsed_time
 5. set_no
 6. game_no
 7. point_no
 8. p1_sets
 9. p2_sets
10. p1_games
11. p2_games
12. p1_score
13. p2_score
14. server
15. serve_no
16. point_victor
17. p1_points_won
18. p2_points_won
19. game_victor
20. set_victor
21. p1_ace
22. p2_ace
23. p1_winner
24. p2_winner
25. winner_shot_type
26. p1_double_fault
27. p2_double_fault
28. p1_unf_err
29. p2_unf_err
30. p1_net_pt
31. p2_net_pt
32. p1_net_pt_won
33. p2_net_pt_won
34. p1_break_pt
35. p2_break_pt
36. p1_break_pt_won
37. p2_break_pt_won
38. p1_break_pt_missed
39. p2_break_pt_missed
40. p1_distance_run
41. p2_distance_run
42. rally_count
43. speed_mph
44. serve_width
45. serve_depth
46. return_depth
47. elapsed_time_td
48. elapsed_seconds
49. global_point_idx
50. point_diff
51. p1_rolling_win_rate_5
52. p1_rolling_win_rate_10
53. p1_streak
54. p2_streak
55. p1_streak_prev
56. p2_streak_prev
57. is_p1_serving
58. server_won
59. games_in_set
60. sets_played
61. is_break